In [74]:
from sklearn.feature_selection import SelectKBest, f_classif
import numpy as np
import pandas as pd
diabetes_df = pd.read_csv('diabetes.csv')
heart_df = pd.read_csv('heart.csv')
heart_df = heart_df.loc[heart_df['sex'] == 0]
heart_df = heart_df.drop(['thalach','exang','oldpeak','slope','sex','thal','restecg'],axis=1)
# Apply SelectKBest on the diabetes dataset
k_best = SelectKBest(f_classif, k=3)
diabetes_features = diabetes_df.drop('Outcome', axis=1)
diabetes_labels = diabetes_df['Outcome']
diabetes_features_new = k_best.fit_transform(diabetes_features, diabetes_labels)
selected_features_diabetes = np.array(diabetes_features.columns)[k_best.get_support()]

# Apply SelectKBest on the heart disease dataset
k_best = SelectKBest(f_classif, k=6)
heart_features=heart_df.drop(['target'],axis=1)
heart_labels = heart_df['target']
heart_features_new = k_best.fit_transform(heart_features, heart_labels)
selected_features_heart = np.array(heart_features.columns)[k_best.get_support()]
print(selected_features_diabetes)
print(selected_features_heart)
diabetes_df=diabetes_df[selected_features_diabetes]
heart_df =heart_df[selected_features_heart]

['Glucose' 'BMI' 'Age']
['Age' 'cp' 'trestbps' 'chol' 'fbs' 'ca']


(312, 6)

In [118]:
merged_df = pd.merge(diabetes_df, heart_df, how='outer', on='Age')
merged_df= merged_df.drop_duplicates()
merged_df['target'] = ((merged_df['trestbps'] >= 140) & (merged_df['BMI'] > 25) & (merged_df['Age'] <= 70)).astype(int)
merged_df.loc[merged_df['fbs'] == 1, 'target'] = 1
merged_df.loc[(merged_df['cp'] ==1) & (merged_df['BMI'] > 25 ) & (merged_df['chol'] > 200), 'target'] = 1
merged_df.loc[(merged_df['cp'] ==2) & (merged_df['chol'] > 200), 'target'] = 1
merged_df.loc[(merged_df['ca'] >= 2)&(merged_df['chol'] > 200), 'target'] = 1
np.shape(merged_df)

(1174, 9)

In [120]:
merged_df.isnull().sum()


Glucose       5
BMI           5
Age           0
cp          527
trestbps    527
chol        527
fbs         527
ca          527
target        0
dtype: int64

In [121]:
mean_values = merged_df.mean().astype(int)
merged_df.fillna(mean_values, inplace=True)


In [122]:
np.shape(merged_df)

(1174, 9)

In [123]:
merged_df.head()

,Glucose,BMI,Age,cp,trestbps,chol,fbs,ca,target
0,148.0,33.6,50,1.0,120.0,244.0,0.0,0.0,1
2,148.0,33.6,50,2.0,120.0,219.0,0.0,0.0,1
3,148.0,33.6,50,0.0,110.0,254.0,0.0,0.0,0
9,99.0,35.4,50,1.0,120.0,244.0,0.0,0.0,1
11,99.0,35.4,50,2.0,120.0,219.0,0.0,0.0,1


In [124]:
merged_df.to_csv('PCOS_dataset.csv', index=False)